In [14]:
%%file counterMapper.py
#!/home/ubuntu/anaconda3/bin/python
"""
Mapper menghitung banyak data
Berikut adalah code pada mapper yang sekaligus berfungsi sebagai kombiner yang digunakan untuk melakukan transformasi data.
<data1> <data2> <data3> menjadi <sum(1, 1, 1)>
"""
import sys
"""
Analisa kode berikut untuk menambahkan pemahaman anda!

Mapper yang dibuat terlihat telah menghitung jumlah data tanpa adanya reducer.
Akan tetapi, Cobalah jalankan mapper tanpa reducer di bawah untuk menambahkan pemahaman anda tentang MapReduce.
"""
counter = 0
for line in sys.stdin:
    counter += 1
print(counter)

Overwriting counterMapper.py


In [15]:
%%file counterReducer.py
#!/home/ubuntu/anaconda3/bin/python
"""
Reducer Menghitung banyak data
Berikut adalah code pada reducer untuk menjumlahkan hasil mapper.
<sum(jumlah_mapper_1)> <sum(jumlah_mapper_2)> menjadi <sum(jumlah_mapper_1, jumlah_mapper_2)>
"""
import sys
"""
Analisa kode berikut untuk menambahkan pemahaman anda!
"""
current_sum=0
for line in sys.stdin:
    current_sum += int(line)
print(current_sum)

Overwriting counterReducer.py


In [16]:
%%file cartesianMapper.py
#!/home/ubuntu/anaconda3/bin/python
"""
Mapper Kartesian
Berikut adalah code pada mapper yang berfungsi mentransformasikan data.

<data1> <data2> <data3> 
menjadi 
<(1,1) data1> <(1,2) data1> ... <(1,n) data1> <(2, 1) data1> ... <(n,n) data1> <(1,1) data2> ... <(n,n) datan>

"""
import sys

#membaca file counter yang berisi nilai banyak data
with open("counter",'r') as c:
    data_count = int(c.read())

"""
Analisa kode berikut untuk menambahkan pemahaman anda!
"""
counter = 1
for value in sys.stdin:
    value = int(value)
    for i in range(data_count):
        print("(%d,%d) %d" % (counter, i+1, value))
        print("(%d,%d) %d" % (i+1, counter, value))
    counter +=1

Overwriting cartesianMapper.py


In [17]:
%%file cartesianReducer.py
#!/home/ubuntu/anaconda3/bin/python
"""
Reducer Kartesian
Berikut adalah code pada reducer yang berfungsi mengkalikan data dengan key yang sama.

<(1,1) data1> <(1,1) data2> menjadi <(1,1) data1 * data2> 

"""

"""
Analisa kode berikut untuk menambahkan pemahaman anda!
Kode ini mirip seperti pada kasus wordcount.
"""
import sys
current_key = None
current_sum = 0
for kv in sys.stdin:
    key, value = kv.split(" ")
    value = int(value)
    if current_key == key :
        current_sum *= value
    elif current_key != key:
        if current_key is not None:
            print("%s %d" % (current_key, current_sum))
        current_key = key
        current_sum = value
print("%s %d" % (current_key, current_sum))


Overwriting cartesianReducer.py


In [18]:
!chmod u+x counterReducer.py cartesianMapper.py cartesianReducer.py

In [19]:
#bangkitkan data yang berisi angka 1 - 10
count = 10
with open("numbers.txt", "w") as numbers:
    for i in range(count):
        numbers.write("%d\n" % (i+1))

In [20]:
#jalankan program menghitung banyak data. simpan ke file counter
!cat numbers.txt | python counterMapper.py | sort |python counterReducer.py  > counter
#jalankan program menghitung kartesian product
!cat numbers.txt | python cartesianMapper.py | sort | python cartesianReducer.py

(1,1) 1
(1,10) 10
(1,2) 2
(1,3) 3
(1,4) 4
(1,5) 5
(1,6) 6
(1,7) 7
(1,8) 8
(1,9) 9
(10,1) 10
(10,10) 100
(10,2) 20
(10,3) 30
(10,4) 40
(10,5) 50
(10,6) 60
(10,7) 70
(10,8) 80
(10,9) 90
(2,1) 2
(2,10) 20
(2,2) 4
(2,3) 6
(2,4) 8
(2,5) 10
(2,6) 12
(2,7) 14
(2,8) 16
(2,9) 18
(3,1) 3
(3,10) 30
(3,2) 6
(3,3) 9
(3,4) 12
(3,5) 15
(3,6) 18
(3,7) 21
(3,8) 24
(3,9) 27
(4,1) 4
(4,10) 40
(4,2) 8
(4,3) 12
(4,4) 16
(4,5) 20
(4,6) 24
(4,7) 28
(4,8) 32
(4,9) 36
(5,1) 5
(5,10) 50
(5,2) 10
(5,3) 15
(5,4) 20
(5,5) 25
(5,6) 30
(5,7) 35
(5,8) 40
(5,9) 45
(6,1) 6
(6,10) 60
(6,2) 12
(6,3) 18
(6,4) 24
(6,5) 30
(6,6) 36
(6,7) 42
(6,8) 48
(6,9) 54
(7,1) 7
(7,10) 70
(7,2) 14
(7,3) 21
(7,4) 28
(7,5) 35
(7,6) 42
(7,7) 49
(7,8) 56
(7,9) 63
(8,1) 8
(8,10) 80
(8,2) 16
(8,3) 24
(8,4) 32
(8,5) 40
(8,6) 48
(8,7) 56
(8,8) 64
(8,9) 72
(9,1) 9
(9,10) 90
(9,2) 18
(9,3) 27
(9,4) 36
(9,5) 45
(9,6) 54
(9,7) 63
(9,8) 72
(9,9) 81


In [21]:
!rm counter
!hdfs dfs -rm -r input/cartesian
!hdfs dfs -rm -r output/cartesian
!hdfs dfs -mkdir -p input/cartesian
!hdfs dfs -put numbers.txt input/cartesian

Deleted input/cartesian
Deleted output/cartesian


In [22]:
!hadoop jar /home/ubuntu/hadoop-2.8.5/share/hadoop/tools/lib/hadoop-streaming-2.8.5.jar \
-mapper counterMapper.py \
-reducer counterReducer.py \
-input input/cartesian/numbers.txt \
-output output/cartesian/tmp

#gabung hasil reducer program menghitung banyak data (sebenarnya tidak diperlukan hanya perlu diunduh saja pada master)
!hdfs dfs -getmerge output/cartesian/tmp/* counter
#masukkan hasil perhitungan banyak data ke HDFS
!hdfs dfs -put counter output/cartesian/tmp/counter

!hadoop jar /home/ubuntu/hadoop-2.8.5/share/hadoop/tools/lib/hadoop-streaming-2.8.5.jar \
-mapper cartesianMapper.py \
-reducer cartesianReducer.py \
-input input/cartesian/numbers.txt \
-output output/cartesian/result \
-cacheFile "output/cartesian/tmp/counter#counter"

#perhatikan perintah Hadoop diatas!
#fungsi parameter -cacheFile adalah memberitahu program terdapat sebuah cache file yang dapat dibuka.
#cache file adalah file berukuran kecil yang berisi konfigurasi atau data berukuran kecil lain yang digunakan -
#untuk menunjang kinerja MapReduce.

#cache file disarankan berukuran se-KECIL mungkin karena cara kerja cache file yang disalin ke masing-masing node -
#sehingga dapat mengurangi kinerja MapReduce.

#Dalam kasus ini cacheFile berisi banyak data yang telah dihitung sebelumnya dan hanya berukuran beberapa KB.

19/06/29 05:37:48 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
19/06/29 05:37:48 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
19/06/29 05:37:48 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
19/06/29 05:37:48 INFO mapred.FileInputFormat: Total input files to process : 1
19/06/29 05:37:48 INFO mapreduce.JobSubmitter: number of splits:1
19/06/29 05:37:48 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1258039002_0001
19/06/29 05:37:49 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
19/06/29 05:37:49 INFO mapreduce.Job: Running job: job_local1258039002_0001
19/06/29 05:37:49 INFO mapred.LocalJobRunner: OutputCommitter set in config null
19/06/29 05:37:49 INFO mapred.LocalJobRunner: OutputCommitter is org.apache.hadoop.mapred.FileOutputCommitter
19/06/29 05:37:49 INFO output.FileOutputCommitter: Fil

19/06/29 05:37:50 INFO streaming.PipeMapRed: R/W/S=1/0/0 in:NA [rec/s] out:NA [rec/s]
19/06/29 05:37:50 INFO streaming.PipeMapRed: Records R/W=1/1
19/06/29 05:37:50 INFO streaming.PipeMapRed: MRErrorThread done
19/06/29 05:37:50 INFO streaming.PipeMapRed: mapRedFinished
19/06/29 05:37:50 INFO mapreduce.Job: Job job_local1258039002_0001 running in uber mode : false
19/06/29 05:37:50 INFO mapreduce.Job:  map 100% reduce 0%
19/06/29 05:37:50 INFO mapred.Task: Task:attempt_local1258039002_0001_r_000000_0 is done. And is in the process of committing
19/06/29 05:37:50 INFO mapred.LocalJobRunner: 1 / 1 copied.
19/06/29 05:37:50 INFO mapred.Task: Task attempt_local1258039002_0001_r_000000_0 is allowed to commit now
19/06/29 05:37:50 INFO output.FileOutputCommitter: Saved output of task 'attempt_local1258039002_0001_r_000000_0' to hdfs://localhost:8020/user/ubuntu/output/cartesian/tmp/_temporary/0/task_local1258039002_0001_r_000000
19/06/29 05:37:50 INFO mapred.LocalJobRunner: Records R/W=1/1 >

19/06/29 05:38:03 INFO mapred.MapTask: Finished spill 0
19/06/29 05:38:03 INFO mapred.Task: Task:attempt_local933574709_0001_m_000000_0 is done. And is in the process of committing
19/06/29 05:38:03 INFO mapred.LocalJobRunner: Records R/W=10/1
19/06/29 05:38:03 INFO mapred.Task: Task 'attempt_local933574709_0001_m_000000_0' done.
19/06/29 05:38:03 INFO mapred.Task: Final Counters for attempt_local933574709_0001_m_000000_0: Counters: 22
	File System Counters
		FILE: Number of bytes read=134050
		FILE: Number of bytes written=501319
		FILE: Number of read operations=0
		FILE: Number of large read operations=0
		FILE: Number of write operations=0
		HDFS: Number of bytes read=25
		HDFS: Number of bytes written=0
		HDFS: Number of read operations=22
		HDFS: Number of large read operations=0
		HDFS: Number of write operations=1
	Map-Reduce Framework
		Map input records=10
		Map output records=200
		Map output bytes=1860
		Map output materialized bytes=2266
		Input split bytes=113
		Combine i

In [24]:
!hdfs dfs -cat output/cartesian/result/*

(1,1) 1	
(1,10) 10	
(1,2) 2	
(1,3) 3	
(1,4) 4	
(1,5) 5	
(1,6) 6	
(1,7) 7	
(1,8) 8	
(1,9) 9	
(10,1) 10	
(10,10) 100	
(10,2) 20	
(10,3) 30	
(10,4) 40	
(10,5) 50	
(10,6) 60	
(10,7) 70	
(10,8) 80	
(10,9) 90	
(2,1) 2	
(2,10) 20	
(2,2) 4	
(2,3) 6	
(2,4) 8	
(2,5) 10	
(2,6) 12	
(2,7) 14	
(2,8) 16	
(2,9) 18	
(3,1) 3	
(3,10) 30	
(3,2) 6	
(3,3) 9	
(3,4) 12	
(3,5) 15	
(3,6) 18	
(3,7) 21	
(3,8) 24	
(3,9) 27	
(4,1) 4	
(4,10) 40	
(4,2) 8	
(4,3) 12	
(4,4) 16	
(4,5) 20	
(4,6) 24	
(4,7) 28	
(4,8) 32	
(4,9) 36	
(5,1) 5	
(5,10) 50	
(5,2) 10	
(5,3) 15	
(5,4) 20	
(5,5) 25	
(5,6) 30	
(5,7) 35	
(5,8) 40	
(5,9) 45	
(6,1) 6	
(6,10) 60	
(6,2) 12	
(6,3) 18	
(6,4) 24	
(6,5) 30	
(6,6) 36	
(6,7) 42	
(6,8) 48	
(6,9) 54	
(7,1) 7	
(7,10) 70	
(7,2) 14	
(7,3) 21	
(7,4) 28	
(7,5) 35	
(7,6) 42	
(7,7) 49	
(7,8) 56	
(7,9) 63	
(8,1) 8	
(8,10) 80	
(8,2) 16	
(8,3) 24	
(8,4) 32	
(8,5) 40	
(8,6) 48	
(8,7) 56	
(8,8) 64	
(8,9) 72	
(9,1) 9	
(9,10) 90	
(9,2) 18	
(9,3) 27	
(9,4) 36	
(9,5) 45	
(9,6) 54	
(9,7) 63	
(9,8) 72	
(9,9) 81	


<h2>Cobalah ganti jumlah data menjadi sebanyak mungkin (jangan terlalu banyak) dan amati lama eksekusi program!!</h2>